In [ ]:
import torch
from diffusers import (
    AutoencoderKL,
    EulerAncestralDiscreteScheduler,
    StableDiffusionImg2ImgPipeline
)
from diffusers.utils import load_image, make_image_grid
from ip_adapter import IPAdapter

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    print ("CUDA & MPS devices not found.")

print("Testing torch device")
torch.ones(2, device=device) 

Variational Autoencoders

In [ ]:
vae = AutoencoderKL.from_pretrained(
    "stabilityai/sd-vae-ft-mse",
    torch_dtype=torch.float16,
    safety_checker=None
)

Stable Diffusion Pipeline

In [ ]:
# stable diffusion pipeline
# pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
#     "SG161222/Realistic_Vision_V4.0_noVAE",
#     torch_dtype=torch.float16,
#     safety_checker=None
# )

pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix",
    vae=vae,
    torch_dtype=torch.float16,
    safety_checker=None
)
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to(device)

Downloading the ip_adapter and the image_encoder

In [ ]:
# from huggingface_hub import hf_hub_download

# hf_hub_download(repo_id="h94/IP-Adapter", filename="models/ip-adapter_sd15.bin", local_dir_use_symlinks=False, local_dir="./")
# hf_hub_download(repo_id="h94/IP-Adapter", filename="models/image_encoder/config.json", local_dir_use_symlinks=False, local_dir="./")
# hf_hub_download(repo_id="h94/IP-Adapter", filename="models/image_encoder/pytorch_model.bin", local_dir_use_symlinks=False, local_dir="./")

IP Adapter for Image Prompt

In [ ]:
# load ip-adapter
ip_model = IPAdapter(pipeline, "models/image_encoder/", "models/ip-adapter_sd15.bin", device)

Input image

In [ ]:
# size = (608, 768)
from PIL import Image

image1 = Image.open("../input/IMG_0521.png")
image2 = load_image("../outputs/beach2.png")

image2 = image2.resize(image1.size)
image1

In [ ]:
pil_image=image1
prompt=image2
negative_prompt="nsfw, bad quality, bad anatomy, worst quality, low quality, low resolutions, extra fingers, blur, blurry, ugly, wrongs proportions, watermark, image artifacts, lowres, ugly, jpeg artifacts, deformed, noisy image"
num_samples=4
num_inference_steps=55
guidance_scale=6

outputs = ip_model.generate(
    pil_image=pil_image,
    image=prompt,
    num_samples=num_samples,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    negative_prompt=negative_prompt,
    prompt="A good beach and a couple ahead"
)

In [ ]:
make_image_grid(outputs, rows=1, cols=num_samples)